In [10]:
import random
import typing
from slippi_db import upload_lib

In [2]:
prod_meta = upload_lib.get_db('prod', 'meta')

In [3]:
meta = list(prod_meta.find({}))

In [16]:
meta_by_key = {info['key']: info for info in meta}

In [5]:
meta[0]

{'_id': ObjectId('62040d02bea2e42533e20205'),
 'invalid': True,
 'reason': 'parse error @0x0: failed to fill whole buffer',
 'key': 'd41d8cd98f00b204e9800998ecf8427e'}

In [6]:
meta[1]

{'_id': ObjectId('62040d02bea2e42533e20206'),
 'startAt': '2020-07-12T22:49:07Z',
 'lastFrame': 15335,
 'playedOn': 'dolphin',
 'slippi_version': [3, 6, 0],
 'num_players': 2,
 'players': [{'port': 1, 'character': 22, 'type': 0, 'name_tag': ''},
  {'port': 2, 'character': 18, 'type': 0, 'name_tag': ''}],
 'stage': 31,
 'timer': 480,
 'is_teams': False,
 'winner': 1,
 'key': '409a3c21fe0d03c2b4195bb7f40ce52f'}

In [13]:
class Settings(typing.NamedTuple):
    num_invalid: int
    rest: int

In [21]:
def gather_keys(settings: Settings) -> typing.List[str]:
    infos = []
    
    # invalid
    invalid = list(filter(lambda info: info.get('invalid', False), meta))
    infos.extend(random.sample(invalid, settings.num_invalid))

    valid = list(filter(lambda info: not info.get('invalid', False), meta))
    infos.extend(random.sample(valid, settings.rest))

    keys = [info['key'] for info in infos]
    return keys

In [22]:
compression_test_settings = Settings(
    num_invalid=0,
    rest=10,
)

compression_test_keys = gather_keys(compression_test_settings)

In [30]:
old_env = 'prod'

def make_new_env(new_env: str, keys: typing.List[str]):
    for db_name in [upload_lib.SLP]:
        old_db = upload_lib.get_db(old_env, db_name)
        new_db = upload_lib.get_db(new_env, db_name)
        
        infos = []
        for key in keys:
            info = old_db.find_one({'key': key})
            del info['_id']
            infos.append(info)

        new_db.insert_many(infos)
    
    for key in keys:
        source = dict(
            Bucket=upload_lib.bucket.name,
            Key='/'.join([old_env, upload_lib.SLP, key]),
        )
        
        upload_lib.bucket.copy(
            CopySource=source,
            Key='/'.join([new_env, upload_lib.SLP, key]),
        )

In [31]:
make_new_env('compression_test', compression_test_keys)

In [35]:
upload_lib.nuke_replays('compression_test', 'none')

No objects to delete.
